# Description

In thiks project I am going to create spell checking system and evaluate its performance.

For the dataset I am using `holbrook.txt` file.

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .

*Note: We are using NLTK libraries. It should not be necessary to use other libraries, but currenly I am focusing on NLTK.*

## Task 1

We are writting a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. 
In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [ ]:
from google.colab import files
import io
uploaded = files.upload()

#To solve the error "initial_value must be unicode or None, not str" decode is used
lines = io.StringIO(uploaded['holbrook.txt'].decode('utf-8')).readlines()

In [ ]:
lines

In [ ]:
import nltk as nk
nk.download('all')

In [ ]:
data = []

#Seprating the words into corrected and originals. Also stroing the indexes of the words.
for i in range(len(lines)):
    if "|" in lines[i]:
        lines_in_arr = nk.word_tokenize(lines[i].encode('ascii', 'ignore'))
        initial_data = {
            'original': list(lines_in_arr),
            'corrected': list(lines_in_arr),
            'indexes': []
        }
        for j in initial_data['original']:
          if("|" in j) == True:
            index_correction = initial_data['original'].index(j)
            correction_values = j.split("|")
            initial_data['corrected'][index_correction] = correction_values[1]
            initial_data['original'][index_correction] = correction_values[0]            
            initial_data['indexes'].append(index_correction)
        data.append(initial_data)                
    else:
        initial_data = {
            'corrected' : nk.word_tokenize(lines[i].encode('ascii', 'ignore')),
            'original': nk.word_tokenize(lines[i].encode('ascii', 'ignore'))
        }
        data.append(initial_data)

In [ ]:
print(data[164])

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [ ]:
test = data[:100]
train = data[100:]

## **Task 2**: 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Using `Counter` to implement this so it may be called many times*

In [ ]:
#Creating filtering training data
filtering_training_data = []
for tr in range(len(train)):
    filtering_training_data = filtering_training_data + train[tr]["corrected"]
print(filtering_training_data)

In [ ]:
from collections import Counter

#Calculating Unigram for a word.
def unigram(word):   
    count_of_word = Counter(w.lower() for w in filtering_training_data)
    return count_of_word[word.lower()]
    
#Calculating the probability of a word.
def prob(word):
    count = unigram(word)
    total = 0
    for w in filtering_training_data:        
        total += 1
    return float(count)/total

In [ ]:
print(unigram("ME"))
print(unigram("the"))
print(prob("the"))

## **Task 3**: 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [ ]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

Writting a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Using the built-in NLTK function `edit_distance`*

In [ ]:
#Finding the distance between wrong and the predicted correct word(s).
def get_candidates(token):
    word_distance = []
    all_word_distance = []
    min_distance = 0
    result = []
    all_unqiue_words = set(filtering_training_data)
    for y in all_unqiue_words:
        word_distance = {
            "word":  y,
            "distance": edit_distance(y, token)
        }
        all_word_distance.append(word_distance)
        if min_distance == 0:
            min_distance = word_distance['distance']
        elif word_distance['distance'] <= min_distance:
            min_distance = word_distance["distance"]

    for j in all_word_distance:
        if j["distance"] == min_distance:
            result.append(j["word"])
    return result
        
# Test your code as follows
get_candidates("minde")

## Task 4:

Writting a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [ ]:
#Correcting the word in a sentence.
def correct(sentence):
    split_sentence = sentence
    all_predicted_words = []
    prob_all_predicted_words = []
    mispelled_words = [i for i in split_sentence if i not in set(filtering_training_data)]
    
    for word in mispelled_words:
        predicted_words = get_candidates(word)
        for pw in predicted_words:
            predicted_word = {
                "word": pw,
                "probability": prob(pw)
            }
            all_predicted_words.append(predicted_word)
        for apw in all_predicted_words:
            prob_all_predicted_words.append(apw["probability"])
        max_word_prob = max(prob_all_predicted_words)
        split_sentence[split_sentence.index(word)] = [t["word"].lower() for t in all_predicted_words if t["probability"] == max_word_prob][0]
    return split_sentence

In [ ]:
print(correct(["this","whitr","ct"]))
print(correct(["Enfield"]))

## **Task 5**: 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [ ]:
original_test_data = []
corrected_test_data = []
check_correct_words = []
correct_counter = []
incorrect_counter = []
for te in range(len(test)):
    if 'original' in test[te]:
        original_test_data = original_test_data + test[te]['original']
for te in range(len(test)):
  corrected_test_data = corrected_test_data + test[te]['corrected']

In [ ]:
print(len(corrected_test_data))
print(len(original_test_data))
print(corrected_test_data)

In [ ]:
#Calculating the accuracy of our system.
def accuracy(test):  
    check_correct_words = correct(original_test_data)    
    for cword in range(len(check_correct_words)):
      if check_correct_words[cword] == corrected_test_data[cword]:
        correct_counter.append(check_correct_words[cword])
      else:
        incorrect_counter.append(check_correct_words[cword])
    calc_accuracy = float(len(correct_counter))/len(check_correct_words)
    
    print("TRUE ",len(correct_counter))
    print("FALSE ",len(incorrect_counter))
    print(incorrect_counter)
    print(set(correct_counter))
    return calc_accuracy

print(accuracy(test))

## **Task 6:**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lecutures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)



**In the current algorithm:**

We are sending the original test data (words containing correct and misspelled) into a correct function, which will filter out all misspelled word and then run the loop to calculate the distance of misspelled word with nearly correct words in data set.  After getting the list of maybe corrected words we will take out their probability using unigram and replace the misspelled word with the max probability of correct word. If, there is a tie between the distance and probability of two words we are explicitly taking the first correct word in the result array. Once we get the correct sentence, we are comparing the result with the test correct sentences to find out the accuracy of an algorithm. The accuracy of the first algorithm is **78.69%.** 

I think, there is some flaw in the first algorithm. To try to improve the accuracy and improve my first algorithm. I found out some other effective algorithms (which are taught in the class) which are: 

  **1. Bigram:**

  -	Many times words repeat themselves in pairs like (good, work), (they, go) etc. If we need to identify such pairs of words which will help us in sentiment analysis. To do this, we will generate such word pairs from the given corpus or sentences preserving their sequences. These pairs are called bigrams. 
  
  -	In python nltk, we have an inbuilt function for same. I.e. nltk.bigrams() and to calculate probability we can use another inbuild function of nltk function ConditionalFreqDist, using this we can find the probability.
 
  **2.	HMM:**

  -	It is mainly used for the analysis of sequence data and is widely used in many fields like speech recognition, gene prediction, pos tagging, text translation, sequence prediction and time series analysis.
  
  -	It is the most efficient but it takes start probabilities (as per my understating which is given by the user) so we cannot use this model.

  **3.	POS Tagging:**
  
  -	Parts of speech (POS) tagging means that distribution grammatical classes i.e. appropriate elements of speech tags to every word in a very natural language sentence.  
  
  -	The process takes a word or a sentence as input, assigns a POS tag to the word or to each word within the sentence, and produces the tagged text as output. 
  
  -	POS tagging can be employed in Text to Speech (TTS) applications, information retrieval, parsing, 
information extraction, linguistic research for corpora and also will be used as an intermediate step for higher level NLP tasks such as parsing, semantic analysis, translation, and many additional, which 
makes POS tagging a necessary function for advanced NLP applications. 

  -	POS tagging is useful for syntactical parsing as taggers scale back ambiguity from the parser's input sentence, which makes parsing quicker by creating the computational drawback smaller, and the result less ambiguous. It also resolves some ambiguities that are not addressed by the syntactical parser’s language model.

  **4.	Smoothing Methods:**
  
  -	In add-one smoothing, we add values in the numerator and denominator so that while calculating the probability of some word we will not get the zero probability even if the word is not available in the given sentence or corpus. 

### Modified Algorithm:

-	After checking through the process of all the above different algorithms/methods I decide to go ahead with the POS Tagging because I believe that it will be helpful in increasing the accuracy as is it tagging each word with nouns, verbs, determinant, cardinal number etc. so it will beneficial for us to ignore the cardinal numbers and pronouns. 

-	***For example***, in my first algorithm if a sentence contains a word “THRUSH” it will be converted to “house” as the house will have the maximum probability. But that should not happen, as “THRUSH” is a pronoun and has special meaning in the sentence, while the replaced word house don’t make any meaning in the sentence. Similarly, cardinal number “48” is getting converted into “4” maybe cardinal number has some meaning in the sentence and while converting that we are losing the correctness and meaning of the sentence.  

-	In addition to, in first algorithm we are not aware of the thing that the word is converting to which part of speech, but in the modified version we can check that out too.

For testing this and to check the accuracy we have done the pos tagging on test original data set and we are passing it to the unchanged correct function. Once the words gets corrected we are checking if the word has either “CD” or “NNP” tag. If they have we are ignoring them (assuming them as a correct). Than calculating the accuracy accordingly by using the formula i.e. no of correct word prediction/total number of words. 

Hence, after using POS tagging we are getting an accuracy of **90.05 %**.


## **Task 7:**

Repeating the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [ ]:
tpd = []
for tr in range(len(test)):  
  x = " ".join(test[tr]["original"])
  tpd = tpd + nk.pos_tag(nk.word_tokenize(x))

In [ ]:
print(tpd)

In [ ]:
cx = [wx for wx, px in tpd]
print(len(cx))

In [ ]:
correct_counter = []
incorrect_counter = []
def accuracy1(test):
    check_correct_words = correct(cx)
    for cword in range(len(check_correct_words)):      
      if ((tpd[cword][1] == 'CD') | (tpd[cword][1] == 'NNP') | (tpd[cword][0] == corrected_test_data[cword])):
        correct_counter.append(tpd[cword][0])  
      else:
        incorrect_counter.append(tpd[cword][0])
    calc_accuracy = float(len(correct_counter))/len(check_correct_words)
    print("TRUE ",len(correct_counter))
    print("FALSE ",len(incorrect_counter))
    print(incorrect_counter)
    print(correct_counter)
    print(len(check_correct_words))
    return calc_accuracy

In [ ]:
print(accuracy1(test))